## FCS Lab 2 Submission Report

* Name of Student(s): James Raphael Tiovalen
* Student ID(s): 1004555

In [1]:
import base64
import requests

def XOR(a, b):
    r = b""
    for x, y in zip(a, b):
        r += (x ^ y).to_bytes(1, "big")
    return r

class Client:
    def __init__(self, endpoint, uid):
        self.endpoint = endpoint
        self.uid = str(uid).lower().strip()

    def post(self, url, data=None):
        r = requests.post(url, json=data).json()
        if not r["success"]:
            print("Warning: something might be wrong with the server")
            print("If you don't think is your mistake, please report it!")
        return r

    def get_story_cipher(self):
        url = self.endpoint + "/story"
        return requests.get(url).json()

    def post_story_plaintext(self, solution):
        url = self.endpoint + "/story"
        solution = str(solution).lower().strip()
        data = {"solution": solution}
        return self.post(url, data)

    def get_score_msg_cipher(self):
        url = self.endpoint + "/score"
        data = {"request": "get_msg", "id": self.uid}
        return self.post(url, data)

    def submit_score_msg_cipher(self, cipher_base64):
        url = self.endpoint + "/score"
        data = {"request": "decrypt_msg", "id": self.uid, "cipher": cipher_base64}
        return self.post(url, data)

    def base64_encode_bytes(self, byte_array):
        return str(base64.b64encode(byte_array))[2:-1]

    def base64_decode_bytes(self, base64_string):
        return base64.b64decode(base64_string)

In [2]:
endpoint = "http://35.197.130.121"
uid = "1004555"

client = Client(endpoint, uid)

## Part I: Story - Substitution Cipher

1. GET the cipher for the story
2. Crack this with frequency analysis
3. POST it back to the server to check (example is provided below)

If the response contains `'solution_correct': 'correct'`, then your decryption is correct. Otherwise, a distance will be provided to let you know how far off you are. If you are off by a tiny bit (say, 1 or 2), you can check things like line-ending, extra space at start/end etc. The verification is not case sensitive.

In [3]:
story_cipher = client.get_story_cipher()["cipher"]
print("story_cipher:", story_cipher[:50], "...")

story_cipher: MXQJ YI IOCFXEWUQH. VEH Q BEEEEEDW, BEEEDW JYCU Y  ...


In [4]:
# Example POSTing a string back to the server
client.post_story_plaintext("random")
# A distance is provided for you to check how close you are

{'distance': '3353',
 'hint': 'it is a substitution cipher; it is obvious when correct',
 'solution_correct': 'wrong',
 'success': True}

In [5]:
import string

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
decryptor = len(string.printable[36:62]) - 16
shifted_printables = string.printable[36:62][decryptor:] + string.printable[36:62][:decryptor]

# This final remapping was obtained via naive remapping using English's relative letter frequency order and some additional manual reworking
FINAL_DECRYPTION_REMAPPING = dict(list(zip(ALPHABET, shifted_printables)))
print(FINAL_DECRYPTION_REMAPPING)

{'A': 'K', 'B': 'L', 'C': 'M', 'D': 'N', 'E': 'O', 'F': 'P', 'G': 'Q', 'H': 'R', 'I': 'S', 'J': 'T', 'K': 'U', 'L': 'V', 'M': 'W', 'N': 'X', 'O': 'Y', 'P': 'Z', 'Q': 'A', 'R': 'B', 'S': 'C', 'T': 'D', 'U': 'E', 'V': 'F', 'W': 'G', 'X': 'H', 'Y': 'I', 'Z': 'J'}


In [6]:
# You can also load solution from a text file
with open("./solution.txt", "r") as file:
    PART_1_SOLUTION = file.read()
part_1_result = client.post_story_plaintext(PART_1_SOLUTION)
print(part_1_result)
assert part_1_result["solution_correct"] == "correct"

{'distance': '0', 'solution_correct': 'correct', 'success': True}


## Part II: Changing the Score Message - OTP

In [7]:
response = client.get_score_msg_cipher()
print(response)

{'cipher': 'q9JZfVmaUTP6yY+H/woZfoYLsPSClAqQd/LMCvOL/msbSz5jmNB4+9Ni86s=', 'hint': 'it is a OTP, you will not be able to guess it, find a way to edit the message without the OTP key', 'success': True}


In [8]:
cipher = client.base64_decode_bytes(response["cipher"])
print(cipher)

b'\xab\xd2Y}Y\x9aQ3\xfa\xc9\x8f\x87\xff\n\x19~\x86\x0b\xb0\xf4\x82\x94\n\x90w\xf2\xcc\n\xf3\x8b\xfek\x1bK>c\x98\xd0x\xfb\xd3b\xf3\xab'


In [9]:
encoded_cipher = client.base64_encode_bytes(cipher)
response = client.submit_score_msg_cipher(encoded_cipher)
target_string = f'Student ID {uid} gets a total of 9 points!'.encode()

In [10]:
# Cancel off the original message due to XOR's self-inverse property.
# This way, we are able to modify the original plaintext message and compromise its integrity without even knowing the key/OTP used by the server.
def hax():
    return XOR(cipher, XOR(response["plaintext"].encode(), target_string))

new_cipher = hax()
encoded_new_cipher = client.base64_encode_bytes(new_cipher)
client.submit_score_msg_cipher(encoded_new_cipher)

{'plaintext': 'Student ID 1004555 gets a total of 9 points!', 'success': True}